In [4]:
import tensorflow as tf
from transformers import AutoTokenizer
from transformers import TFGPT2LMHeadModel #tensorflow GPT2 LM
import numpy as np
import torch
import os
from tqdm import tqdm  # 진행률 표시를 위한 tqdm 라이브러리
import evaluate

# 모델과 토크나이저 로드
model_name = 'skt/kogpt2-base-v2'  # 예시 모델, 필요에 따라 다른 모델로 변경 가능
model = TFGPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2', from_pt=True)
tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2')

InternalError: cudaGetDevice() failed. Status: CUDA driver version is insufficient for CUDA runtime version

: 

In [ ]:
def calculate_perplexity(text, model, tokenizer):
    # 텍스트를 토큰화하고 텐서로 변환
    encodings = tokenizer(text, return_tensors='tf')
    max_length = model.config.n_positions  # 모델의 최대 입력 길이
    stride = 512

    # Perplexity를 계산하기 위한 로그-확률을 저장할 변수
    log_likelihood = 0.0
    nlls = []

    # 긴 텍스트를 처리하기 위해 슬라이딩 윈도우 방식 사용
    for i in range(0, encodings.input_ids.shape[1], stride):
        begin_loc = max(i + stride - max_length, 0)
        end_loc = min(i + stride, encodings.input_ids.shape[1])
        trg_len = end_loc - i  # 실제 대상 길이

        input_ids = encodings.input_ids[:, begin_loc:end_loc]
        target_ids = input_ids * 0  # 모든 위치에 대해 loss 계산을 위해 0으로 초기화
        target_ids[:, :-trg_len] = -100  # 실제 토큰 위치만 계산

        with tf.GradientTape() as tape:
            tape.stop_recording()
            outputs = model(input_ids, labels=target_ids)
            neg_log_likelihood = outputs.loss * trg_len  # 배치에 대한 NLL
            nlls.append(neg_log_likelihood)

    # 전체 텍스트에 대한 평균 NLL 계산
    total_nll = tf.reduce_sum(tf.stack(nlls))
    total_length = tf.reduce_sum([tf.maximum(tf.shape(input_ids)[1] - 1, 0) for input_ids in encodings.input_ids])

    # Perplexity 계산
    perplexity = tf.exp(total_nll / total_length)
    return perplexity.numpy()

# 텍스트 예시
text = "이제 겨울이 다가옵니다. 추운 날씨에는"

# Perplexity 계산
ppl = calculate_perplexity(text, model, tokenizer)
print(f"Perplexity: {ppl}")
